# PyJSG -- JSON Schema Grammar Bindings for Python
Translate [JSON Schema Grammar](http://github.com/ericprud/jsg) into Python objects.

This tool generates Python 3 objects that represent the JSON objects defined in a JSG schema.  It uses the [Python Typing library](https://docs.python.org/3/library/typing.html) to add type hints to Python IDE's and includes a library to validate the python objects against the library definitions.

[![Pyversions](https://img.shields.io/pypi/pyversions/PyJSG.svg)](https://pypi.python.org/pypi/PyJSG)

[![PyPi](https://version-image.appspot.com/pypi/?name=PyJSG)](https://pypi.python.org/pypi/PyJSG)

## History
* 0.5.3 -- Simplified Logger - now uses StringIO instead of MemLogger
* 0.5.4 -- Runs with python 3.6 and python 3.7.0b3
* 0.6.0 -- Beginning of cleanup and jupyter documentation


## Testing Framework
The code below allows us to compile a JSG definition into Python and then execute the python classes against sample JSON.  One of the key benefits of PyJSG -- the formally typed python classes, which allows type checking in an IDE -- is not shown here.

In [79]:
from types import ModuleType
from io import StringIO

from pyjsg.jsglib.jsg import loads, JSGException, Logger
from pyjsg.parser_impl.generate_python import parse

In [82]:
class TestModule:
    def __init__(self, jsg: str, test_name: str, print_python: bool=False) -> None:
        """ Generate python from jsg and compile it into module named test_name """
        python = parse(jsg, test_name)
        if print_python:
            print(python)
        spec = compile(python, test_name, 'exec')
        self._module = ModuleType(test_name)
        exec(spec,self._module.__dict__)

    def test(self, *jsons: str) -> None:
        """ Test list of json strings against jsg specification """
        npass = nfail = 0
        for i, json in enumerate(jsons):
            try:
                obj = loads(json, self._module)
            except ValueError as v:
                print(f"String {i+1}: Error: {v}")
                nfail += 1
                continue
            except JSGException as v:
                print(f"String {i+1}: Exception: {v}")
                nfail += 1
                continue
            logfile = StringIO()
            logger = Logger(logfile)
            if not obj._is_valid(logger):
                print(f"String {i+1}: {logfile.getvalue()}")
                nfail += 1
        print(f"{i+1} tests, {nfail} failures")
        print()

## Examples

**Example 1**: A JSON document that must have exactly one property named "status" whose value must be one of "ready" or "pending"

In [83]:
jsg = """
doc {status: ("ready"|"pending")}
"""

# Passing JSON documents
e1pass = """
{"status": "pending"}
"""
e2pass = """
{"status": "ready"
}
"""

# Fails because of "state" attribute
e1f1 = """
{"status": "pending",
 "state": "new"
}
"""

# Fails because of unknown value
e1f2 = """
{"status": "complete"}
"""

# Fails because "statis" attribute
e1f3 = """
{"statis": "pending"}
"""

# Fails because of missing "state"
e1f4 = """
{}
"""
TestModule(jsg, "ex1", print_python=False).test(e1pass, e2pass, e1f1, e1f2, e1f3, e1f4)

String 3: Error: Unknown attribute: state=new
String 4: doc: Type mismatch for status. Expecting: <class 'ex1._Anon1'> Got: <class 'str'>

String 5: Error: Unknown attribute: statis=pending
String 6: doc: Missing required field: status

6 tests, 4 failures



**Example 2**: Define a JSON document that must have at least two properties, `"street"` represented as a JSON string a `"number"` represented as a JSON number and an optional `"state"`.  The trailing comma on `state` indicates an "open" spec -- additional tags are allowed

In [85]:
jsg = """
addr { street:@string,
       number:@int,
       state:@string?,
      }
"""

# A vanilla example
e2p1 = """
{ "street": "South Fanning",
  "number": 608,
  "state": "ID",
  "zip": 83401
}"""

# Omitting the state is ok
e2p2 = """
{ "street": "1st Street",
  "number": 1217
}"""

# Non-numeric number fails
e2f2 = """
{ "street": "1st Street",
  "number": "17a"
}
"""

# Missing street fails
e2f3 = """
{ "location": "South Fanning",
  "number": 608,
  "state": "ID",
  "zip": 83401
}"""
TestModule(jsg, "ex2").test(e2p1, e2p2, e2f2, e2f3)

String 3: Error: Invalid Integer value: "17a"
String 4: addr: Missing required field: street

4 tests, 2 failures



## Directives
### The `.TYPE` directive:

1. Names a unique property that identifies the JSG object being represented
2. (Optional) lists one or more production types that do not use the `.TYPE` discriminator

#### Syntax
`.TYPE <type> [ - <type> [<type>...]] ;`

#### No type directive

In [52]:
jsg = 'doc { a:. }'

t1pass = '{"a":"hello"}'
t1fail = '{"type": "doc", "a":"hello"}'

TestModule(jsg, "test1").test(t1pass, t1fail)

String 2: Error: Unknown attribute: type=doc
2 tests, 1 failures



#### Type directive
The following example defines two JSG types, `doc` and `id`.  `doc` has a single element of any type named 'a'.  'id' has a single element named 'e' that must be a JSON number.

In [53]:
jsg = """
.TYPE type;

doc {a:.}
id {e: @int}
"""
t2p1 = '{"type": "doc", "a": "hello"}'
t2p2 = '{"type": "doc", "a": {"type": "id", "e": 42}}'

# Fails due to unrecognized type
t2f1 = '{"type": "dac", "a": "hello"}'

# Passes - schema allows nested types
t2p3 = '{"type": "doc", "a":{"type": "doc", "a":0}}'

TestModule(jsg, "test2").test(t2p1, t2p2, t2f1, t2p3)


# Fails for lack of a type on the inner element
t2f2 = '{"type": "doc", "a":{"e":0}}'
TestModule(jsg, "test2").test(t2f2)

# Add a type exception
jsg = """
.TYPE type - id;

doc {a:.}
id {e: @int}
"""

# Test now passes
TestModule(jsg, "test2a").test(t2f2)

String 3: Exception: Unknown type: dac
4 tests, 1 failures

String 1: Exception: Missing type var
1 tests, 1 failures

1 tests, 0 failures

